Script which takes a flight csv file and extracts from a camera trigger event the time, and uses that to find the time of the mission start and end. Then it creates a new flight csv file with only the rows (+ 20 each side) of data collected from the flight during its actual mission, to allow for easier GIS plotting of flight mission routes.

Next the script opens the corresponding MAIA log file and finds which the line the mission started and ended on and creates a new csv file name with the Image File Name / Lat / Long / Altitude as it's only four columns, for use in the AgiSoft program

In [111]:
with open('fly_375.csv') as flightFile:      
    flightFileLines = flightFile.read().split("\n")
    print("Number of lines is {}".format(len(flightFileLines)))
    
camShotLine = []
word = 'SUCCESSED SIMPLE_SHOT'
   
for i,line in enumerate(flightFileLines):
    if word in line: # or word in line.split() to search for full words
        camShotLine.append(i)
        print("Word \"{}\" found in line {}".format(word, i))
        
missionRangeLine = []
startWord = '[L-MIS]WP Mission: received idle vel cmd 3.618762 from app'
endWord = '[L-OSD]Mission finished. now I tell app:'
   
for i,line in enumerate(flightFileLines):
    if startWord in line: # or word in line.split() to search for full words
        missionRangeLine.append(i)
        missStartLine = i
        print("Word \"{}\" found in line {}".format(startWord, i))

for j,line in enumerate(flightFileLines):
    if endWord in line: # or word in line.split() to search for full words
        missionRangeLine.append(j)
        missEndLine = j
        print("Word \"{}\" found in line {}".format(endWord, j))
             
potMatch = len(camShotLine)
times = []
potMatchLines = [flightFileLines[x] for x in camShotLine]

for y in range(potMatch):
    if potMatchLines[y].find("20170927.0")>1:
        times.append(potMatchLines[y][(potMatchLines[y].find("20170927.0")+9):((potMatchLines[y].find("20170927.0")+17))])
    elif potMatchLines[y].find("20170927.1")>1:  
        times.append(potMatchLines[y][(potMatchLines[y].find("20170927.1")+9):((potMatchLines[y].find("20170927.1")+17))])
    elif potMatchLines[y].find("20170927.2")>1:
        times.append(potMatchLines[y][(potMatchLines[y].find("20170927.2")+9):((potMatchLines[y].find("20170927.2")+17))])
        
flightTimes = [(flightFileLines[x]).split(',')[1] for x in camShotLine]
print(times)

missStartFlightTime = (flightFileLines[missStartLine]).split(',')[1]
missFinishFlightTime = (flightFileLines[missEndLine]).split(',')[1]
flightTimeDiffs = [float(missStartFlightTime)-float(flightTimes[0]), float(missFinishFlightTime)-float(flightTimes[0])]
realFlightTimes = [0,0]
for j in range(2):
    timeHolder = (times[0]).split(':')
    for i in range(3):
        timeHolder[i] = float(timeHolder[i])
    timeHolder[2] = timeHolder[2] + flightTimeDiffs[j]
    while timeHolder[2] >= 60:
        timeHolder[2] = timeHolder[2] - 60
        timeHolder[1] = timeHolder[1] + 1
        while timeHolder[1] >= 60:
            timeHolder[1] = timeHolder[1] - 60
            timeholder[0] = timeHolder[0] + 1
    while timeHolder[2] <= 0:
        timeHolder[2] = timeHolder[2] + 60
        timeHolder[1] = timeHolder[1] - 1
        while timeHolder[1] <= 0:
            timeHolder[1] = timeHolder[1] + 60
            timeholder[0] = timeHolder[0] - 1
    realFlightTimes[j] = timeHolder

with open('TestName.log') as maiaFile:      
    maiaFileLines = maiaFile.read().split("\n")
    numLines = len(maiaFileLines)
    maiaFlightTimes = [(maiaFileLines[x]).split('\t')[2] for x in range(numLines-1)]

realFlightTimes[0][1] = 43
realFlightTimes[1][1] = 48
    
fStart = str(int(realFlightTimes[0][0])).zfill(2) + ':' + str(int(realFlightTimes[0][1])).zfill(2) + ':' \
        + str(int(realFlightTimes[0][2])).zfill(2)
fEnd = str(int(realFlightTimes[1][0])).zfill(2) + ':' + str(int(realFlightTimes[1][1])).zfill(2) + ':' \
        + str(int(realFlightTimes[1][2])).zfill(2)

for i in range(numLines-1):
    if maiaFlightTimes[i].startswith(fStart):
        maiaStartLine = i
        print('Mission starts on line:', i)
        
for j in range(maiaStartLine, numLines-1):
    if maiaFlightTimes[j].startswith(fEnd):
        maiaEndLine = j
        print('Mission ends on line:', j)
        
with open('newt4.csv', 'w') as outputFlightFile:
    firstRow = flightFileLines[0] + "\n"
    outputFlightFile.write(firstRow)
    for i in range(missionRangeLine[0]-20, missionRangeLine[1]+20):
        row = flightFileLines[i] + "\n"
        outputFlightFile.write(row)
        
maiaFileNames = [(maiaFileLines[x]).split('\t')[3] for x in range(numLines-1)]
maiaLongs = [(maiaFileLines[x]).split('\t')[7] for x in range(numLines-1)]
maiaLats = [(maiaFileLines[x]).split('\t')[9] for x in range(numLines-1)]
maiaAlts = [(maiaFileLines[x]).split('\t')[11] for x in range(numLines-1)]

with open('newtext14.csv', 'w') as outputMaiaFile:
    firstRow = maiaFileNames[0] + "," + maiaLongs[0] + "," + maiaLats[0] + "," + maiaAlts[0] + "\n"
    outputMaiaFile.write(firstRow)
    for i in range(maiaStartLine, maiaEndLine):
        row = maiaFileNames[i] + "," + '-' + maiaLongs[i] + "," + maiaLats[i] + "," + maiaAlts[i] + "\n"
        outputMaiaFile.write(row)

Number of lines is 11101
Word "SUCCESSED SIMPLE_SHOT" found in line 5445
Word "SUCCESSED SIMPLE_SHOT" found in line 5451
Word "SUCCESSED SIMPLE_SHOT" found in line 5627
Word "SUCCESSED SIMPLE_SHOT" found in line 5806
Word "SUCCESSED SIMPLE_SHOT" found in line 5985
Word "SUCCESSED SIMPLE_SHOT" found in line 6163
Word "SUCCESSED SIMPLE_SHOT" found in line 6342
Word "SUCCESSED SIMPLE_SHOT" found in line 6455
Word "SUCCESSED SIMPLE_SHOT" found in line 6736
Word "SUCCESSED SIMPLE_SHOT" found in line 6915
Word "SUCCESSED SIMPLE_SHOT" found in line 7093
Word "SUCCESSED SIMPLE_SHOT" found in line 7272
Word "SUCCESSED SIMPLE_SHOT" found in line 7450
Word "SUCCESSED SIMPLE_SHOT" found in line 7629
Word "SUCCESSED SIMPLE_SHOT" found in line 7724
Word "[L-MIS]WP Mission: received idle vel cmd 3.618762 from app" found in line 5455
Word "[L-OSD]Mission finished. now I tell app:" found in line 7843
['16:01:55', '16:01:55', '16:02:01', '16:02:07', '16:02:13', '16:02:19', '16:02:25', '16:02:29', '16:02

Script which when given a folder of MAIA images and corresponding log file(s) and the corresponding mission time(s) for these flights will create 2 new subfolders per log file. One to house images taken before and after the mission flight path, and one to house those taken on the mission flight path. The script will then identify which images are to be moved into which sub folder, leaving the logfile(s) and any images that do not appear in the logfile(s) where they are

In [114]:
directory = 'C:/Users/Evan/Anaconda3/.ipynb_checkpoints/TestMover'

import os

fileHolder = []
fileCounter = 0
for file in os.listdir(directory):
    if file.endswith(".txt"):
        fileHolder.append(os.path.join(directory, file))
        fileCounter = fileCounter + 1
        
fileData = []
fileData.append((fileHolder[0], 2, 6))

for i in range(fileCounter):
    newpath1 = directory + '/Flight' + str(i+1) + 'MissionData'  
    newpath2 = directory + '/Flight' + str(i+1) + 'NotMissionData'  
    if not os.path.exists(newpath1):
        os.makedirs(newpath1)
    if not os.path.exists(newpath2):
        os.makedirs(newpath2)
    with open(fileData[i][0]) as file:
        fileLines = file.read().split("\n")
        fileImageNames = [(fileLines[x]).split('\t')[3] for x in range(len(fileLines)-1)]
        for j in range(fileData[i][1], fileData[i][2]):
            os.rename(directory + '/' + fileImageNames[j],  newpath1 + '/' + fileImageNames[j])
        for a in range(1, fileData[i][1]):
            os.rename(directory + '/' + fileImageNames[a],  newpath2 + '/' + fileImageNames[a])
        for b in range(fileData[i][2], len(fileImageNames)):
            os.rename(directory + '/' + fileImageNames[b],  newpath2 + '/' + fileImageNames[b])

Copy of above script, needs edited but up to date copy exists on pen drive

In [130]:
directory = 'C:/Users/Evan/Anaconda3/.ipynb_checkpoints/TestMover'

import os

fileHolder = []
fileCounter = 0
for file in os.listdir(directory):
    if file.endswith(".txt"):
        fileHolder.append(os.path.join(directory, file))
        fileCounter = fileCounter + 1

fileData = []

for i in range(fileCounter):
    newpath1 = directory + '/Flight' + str(i+1) + 'MissionData'  
    newpath2 = directory + '/Flight' + str(i+1) + 'NotMissionData'  
    if not os.path.exists(newpath1):
        os.makedirs(newpath1)
    if not os.path.exists(newpath2):
        os.makedirs(newpath2)
    with open(fileHolder[i]) as file:
        fileLines = file.read().split("\n")
        fileImageNames = [(fileLines[x]).split('\t')[3] for x in range(len(fileLines)-1)]
        fileAltitudes = [(fileLines[x]).split('\t')[12] for x in range(len(fileLines)-1)]
        missionAlts = []
        for k in range(1, len(fileAltitudes)):
            if float(fileAltitudes[k]) > 167 + 50:
                missionAlts.append(k)
        fileData.append((fileHolder[0], min(missionAlts), max(missionAlts)))
        for j in range(fileData[i][1], fileData[i][2]+1):
            os.rename(directory + '/' + fileImageNames[j],  newpath1 + '/' + fileImageNames[j])
        for a in range(1, fileData[i][1]):
            os.rename(directory + '/' + fileImageNames[a],  newpath2 + '/' + fileImageNames[a])
        for b in range(fileData[i][2]+1, len(fileImageNames)):
            os.rename(directory + '/' + fileImageNames[b],  newpath2 + '/' + fileImageNames[b])

Now want a script which takes as input a directory full of flight logs and outputs a text file that has each row corresponding to a specifoc flight file, with columns of File Name / Date / Time of flight start/end / Time of mission start/end

This will go in conjunction to a script much like the intial one above, that gives the MAIA log mission start/end times, but the script need only be given the above text file as its input and not need to be given the appropriate flight log

Script below takes as an input the flight logs from the first few flights, when mission start and finish are not noted in the flight log but the phrase "SUCCESSED SIMPLE_SHOT" is and that has a corresponding real time attached. Will use the speed of the UAV to determine when it has finished flying its mission, when the speed is no longer 2ms-1, and possibly use the altitude to inform this decision also

In [141]:
with open('FLY216_20170718_ES_P2.csv') as flightFile:      
    flightFileLines = flightFile.read().split("\n")
    print("Number of lines is {}".format(len(flightFileLines)))
    
camShotLine = []
word = 'SUCCESSED SIMPLE_SHOT'
   
for i,line in enumerate(flightFileLines):
    if word in line: # or word in line.split() to search for full words
        camShotLine.append(i)
        print("Word \"{}\" found in line {}".format(word, i))
                     
potMatch = len(camShotLine)
potMatchLines = [flightFileLines[x] for x in camShotLine]
times = []

for y in range(potMatch):
    if potMatchLines[y].find("20170927.0")>1:
        times.append(potMatchLines[y][(potMatchLines[y].find("20170927.0")+9):((potMatchLines[y].find("20170927.0")+17))])
    elif potMatchLines[y].find("20170927.1")>1:  
        times.append(potMatchLines[y][(potMatchLines[y].find("20170927.1")+9):((potMatchLines[y].find("20170927.1")+17))])
    elif potMatchLines[y].find("20170927.2")>1:
        times.append(potMatchLines[y][(potMatchLines[y].find("20170927.2")+9):((potMatchLines[y].find("20170927.2")+17))])
        
flightTimes = [(flightFileLines[x]).split(',')[1] for x in camShotLine]

#This is the new code START

missionSpeed = 2
missionBumpiness = 0.25
flightSpeeds = [(flightFileLines[x]).split(',')[27] for x in range(len(flightFileLines)-1)]

def speedChecker(vals):
    cnt = 0
    for i in vals:
        if missionSpeed - missionBumpiness < i < missionSpeed + missionBumpiness:
            cnt = cnt + 1
    if cnt == len(vals):
        return 1
    else: 
        return 0

check = 0
counter = 1
while check == 0:
    vals = [float(flightSpeeds[x]) for x in range(counter, counter+30)]
    counter = counter + 1
    check = speedChecker(vals)
    print(check)
    

        

#This is the new code END need missStartLine and missEndLine
missStartFlightTime = (flightFileLines[missStartLine]).split(',')[1]
missFinishFlightTime = (flightFileLines[missEndLine]).split(',')[1]
flightTimeDiffs = [float(missStartFlightTime)-float(flightTimes[0]), float(missFinishFlightTime)-float(flightTimes[0])]
realFlightTimes = [0,0]
for j in range(2):
    timeHolder = (times[0]).split(':')
    for i in range(3):
        timeHolder[i] = float(timeHolder[i])
    timeHolder[2] = timeHolder[2] + flightTimeDiffs[j]
    while timeHolder[2] >= 60:
        timeHolder[2] = timeHolder[2] - 60
        timeHolder[1] = timeHolder[1] + 1
        while timeHolder[1] >= 60:
            timeHolder[1] = timeHolder[1] - 60
            timeholder[0] = timeHolder[0] + 1
    while timeHolder[2] <= 0:
        timeHolder[2] = timeHolder[2] + 60
        timeHolder[1] = timeHolder[1] - 1
        while timeHolder[1] <= 0:
            timeHolder[1] = timeHolder[1] + 60
            timeholder[0] = timeHolder[0] - 1
    realFlightTimes[j] = timeHolder

with open('TestName.log') as maiaFile:      
    maiaFileLines = maiaFile.read().split("\n")
    numLines = len(maiaFileLines)
    maiaFlightTimes = [(maiaFileLines[x]).split('\t')[2] for x in range(numLines-1)]

realFlightTimes[0][1] = 43
realFlightTimes[1][1] = 48
    
fStart = str(int(realFlightTimes[0][0])).zfill(2) + ':' + str(int(realFlightTimes[0][1])).zfill(2) + ':' \
        + str(int(realFlightTimes[0][2])).zfill(2)
fEnd = str(int(realFlightTimes[1][0])).zfill(2) + ':' + str(int(realFlightTimes[1][1])).zfill(2) + ':' \
        + str(int(realFlightTimes[1][2])).zfill(2)

for i in range(numLines-1):
    if maiaFlightTimes[i].startswith(fStart):
        maiaStartLine = i
        print('Mission starts on line:', i)
        
for j in range(maiaStartLine, numLines-1):
    if maiaFlightTimes[j].startswith(fEnd):
        maiaEndLine = j
        print('Mission ends on line:', j)
        
with open('newt4.csv', 'w') as outputFlightFile:
    firstRow = flightFileLines[0] + "\n"
    outputFlightFile.write(firstRow)
    for i in range(missionRangeLine[0]-20, missionRangeLine[1]+20):
        row = flightFileLines[i] + "\n"
        outputFlightFile.write(row)
        
maiaFileNames = [(maiaFileLines[x]).split('\t')[3] for x in range(numLines-1)]
maiaLongs = [(maiaFileLines[x]).split('\t')[7] for x in range(numLines-1)]
maiaLats = [(maiaFileLines[x]).split('\t')[9] for x in range(numLines-1)]
maiaAlts = [(maiaFileLines[x]).split('\t')[11] for x in range(numLines-1)]

with open('newtext14.csv', 'w') as outputMaiaFile:
    firstRow = maiaFileNames[0] + "," + maiaLongs[0] + "," + maiaLats[0] + "," + maiaAlts[0] + "\n"
    outputMaiaFile.write(firstRow)
    for i in range(maiaStartLine, maiaEndLine):
        row = maiaFileNames[i] + "," + '-' + maiaLongs[i] + "," + maiaLats[i] + "," + maiaAlts[i] + "\n"
        outputMaiaFile.write(row)

Number of lines is 29363
Word "SUCCESSED SIMPLE_SHOT" found in line 4157
Word "SUCCESSED SIMPLE_SHOT" found in line 26079
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


IndexError: list index out of range